In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.nn import weighted_cross_entropy_with_logits
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.data import Dataset
from tensorflow.keras.callbacks import ModelCheckpoint

import os
import re
import glob
import random
import numpy as np
import tensorflow as tf

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
remove = ['\u200e', '[', ']', '(', ')', '\x98', '́', '\r', ';']
replace = {
    '»': '"',
    '«': '"',
    '“': '"',
    '„': '"',
    '...': '…',
    '—': '-',
}
signs = ['.', ',', '"', '…', '-', '\n', '?', '!', ':']
vowels = 'а у о ы и э я ю ё е ь'.split(' ')
consonants = 'б в г д ж з й к л м н п р с т ф х ц ч ш щ ъ'.split(' ')

def preprocess_str(string):
    string = string.lower()
    for x in remove:
        string = string.replace(x, '')
    for key, value in replace.items():
        string = string.replace(key, value)
    string = re.sub(r'[. ]{2,}', '. ', string)
    string = re.sub(r' +', ' ', string)
    return string

def split_to_syllables(text):
    syllables = []
    cur_syl = ''
    v_in_cur_syl = False
    for l in text:
        if l in signs or l == ' ':
            syllables.append(cur_syl)
            syllables.append(l)
            cur_syl = ''
            v_in_cur_syl = False
        else:
            if l in vowels and v_in_cur_syl:
                if cur_syl[-1] in consonants:
                    syllables.append(cur_syl[:-1])
                    cur_syl = cur_syl[-1] + l
                else:
                    syllables.append(cur_syl)
                    cur_syl = l
            elif l in vowels and not v_in_cur_syl:
                v_in_cur_syl = True
                cur_syl += l
            else:
                cur_syl += l
    syllables.append(cur_syl)
    return list(filter(lambda x: x, syllables))

syllabels = set()
all_texts = ''
for path in glob.glob('poems/*.txt'):
    with open(path, 'rb') as f:
        all_texts += preprocess_str(f.read().decode('utf-8'))
syllabels.update(split_to_syllables(all_texts))

corpus = {value: i for i, value in enumerate(sorted(syllabels))}
corpus_inv = {value: key for key, value in corpus.items()}

In [4]:
window_size = 30
batch_size = 64

In [5]:
def read_poem(path):
    with open(path, 'rb') as f:
        text = preprocess_str(f.read().decode('utf-8'))
    syllabels = [corpus[x] for x in split_to_syllables(text)]
    remains = len(syllabels) % (window_size + 1) / (window_size + 1)
    total = len(syllabels) // (window_size + 1)
    chunks = [syllabels[i*(window_size + 1):(i+1)*(window_size + 1)] for i in range(total)]
    if (len(syllabels) - total) / window_size > 0.2:
        chunks.append(syllabels[-window_size-1:])
    return chunks

def read_all_poems():
    chunks = []
    for path in glob.glob('poems/*.txt'):
        chunks.extend(read_poem(path))
    
    ds = Dataset.from_tensor_slices(chunks)
    return ds.map(lambda x: (x[:-1], x[1:])) \
        .shuffle(8096).batch(batch_size, drop_remainder=True)

dataset = read_all_poems()

In [6]:
def build_model(batch_size):
    return Sequential([
        Embedding(len(corpus), 3072, batch_input_shape=[batch_size, None]),
        LSTM(1024, return_sequences=True, stateful=True),
        Dense(len(corpus)),
    ])

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model = build_model(batch_size)
model.compile('adam', loss=loss)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 3072)          8745984   
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          16781312  
_________________________________________________________________
dense (Dense)                (64, None, 2847)          2918175   
Total params: 28,445,471
Trainable params: 28,445,471
Non-trainable params: 0
_________________________________________________________________


In [7]:
%%time 

history = model.fit(
    dataset,
    epochs=25,
    callbacks=[
        ModelCheckpoint(
            filepath=os.path.join('training_checkpoints', 'ckpt_{epoch}'),
            save_weights_only=True,
        )
    ],
)

Epoch 1/25
126/126 [==============================] - 17s 132ms/step - loss: 4.5068
Epoch 2/25
126/126 [==============================] - 16s 129ms/step - loss: 3.7732
Epoch 3/25
126/126 [==============================] - 16s 129ms/step - loss: 3.4099
Epoch 4/25
126/126 [==============================] - 16s 129ms/step - loss: 3.2134
Epoch 5/25
126/126 [==============================] - 16s 130ms/step - loss: 3.0767
Epoch 6/25
126/126 [==============================] - 16s 129ms/step - loss: 2.9675
Epoch 7/25
126/126 [==============================] - 17s 131ms/step - loss: 2.8744
Epoch 8/25
126/126 [==============================] - 16s 128ms/step - loss: 2.7897
Epoch 9/25
126/126 [==============================] - 16s 128ms/step - loss: 2.7101
Epoch 10/25
126/126 [==============================] - 16s 128ms/step - loss: 2.6375
Epoch 11/25
126/126 [==============================] - 16s 128ms/step - loss: 2.5643
Epoch 12/25
126/126 [==============================] - 16s 127ms/step - lo

In [8]:
model = build_model(1)
model.load_weights(tf.train.latest_checkpoint('training_checkpoints'))
model.build(tf.TensorShape([1, None]))

In [9]:
def generate_text(model, start_string):
    num_generate = 500
    input_eval = [corpus[s] for s in split_to_syllables(start_string)]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions /= 1.5
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(corpus_inv[predicted_id])

    return (start_string + ''.join(text_generated))

In [10]:
print(generate_text(model, "ах, как хочется мне осень!\nжелтый брег на теле бренном\n"))

ах, как хочется мне осень!
желтый брег на теле бренном
венков в дарниском опершись,
исполнелась лобзают
мерженье крашее угры,
к теле? легкою счастлив, смерть, близ мухой да разлукаво дни свободны
и слабый битвы сладую тобою,
дремучих озенов бедный царстчучестейтый утедел?
налей своих узпятных предков
потомство нахму богатударный -
встанов льстию,
ехали бишься кликку к тихий подчас
не властным искру
восстаньем ветвей, очи гусарских очках надменная пупусть: с морщиной примутиях достигнутый,
отца шести равнится?
явился.-
а скромной ветхой сенним приятность несчастной?
ухи!. приник святых щальих нама!
тишиной звуклишися,
с дьбою что восстанет, как сей строптиной
уродной младостью ногаюдого желучен
законно молодымиман. -
вперив раз клобудет певцу любовицу.

хвалы, нуйся, закрывшийся инопрекличность,
стыдбыль монахмучим:
европа быстрый поузчий
вблизи мы вам воскримой нежно!
разлугась маратать нет?
перед их ужаса прозрачном,
храбрый в сердечных эливой генерой,
курапу…"
други!" медленно, волно